from abc import ABC, abstractmethod
from enum import Enum
from typing import List

# Enum для представлення стану місць
class SeatStatus(Enum):
    AVAILABLE = "O"
    BOOKED = "X"

# Абстрактний клас спостерігача
class Observer(ABC):
    @abstractmethod
    def update(self):
        pass

# Клас системи бронювання квитків
class TicketBookingSystem:
    def init(self, rows, seats_per_row):
        self.rows = rows
        self.seats_per_row = seats_per_row
        # Створення плану місць у вигляді двовимірного масиву
        self.seating_plan = [[SeatStatus.AVAILABLE] * seats_per_row for _ in range(rows)]
        # Список для зберігання спостерігачів
        self.observers = []

    def attach(self, observer):
        # Додавання спостерігача до списку
        self.observers.append(observer)

    def notify_observers(self):
        # Сповіщення всіх прикріплених спостерігачів про оновлення
        for observer in self.observers:
            observer.update()

    def display_seating_plan(self):
        # Виведення плану місць у консоль
        for row in self.seating_plan:
            print(" ".join(seat.value for seat in row))

    def check_availability(self, row, seat):
        # Перевірка доступності конкретного місця
        if 1 <= row <= self.rows and 1 <= seat <= self.seats_per_row:
            return self.seating_plan[row - 1][seat - 1] == SeatStatus.AVAILABLE
        else:
            raise ValueError("Invalid row or seat number")

    def book_ticket(self, row, seat):
        # Бронювання квитка та сповіщення спостерігачів
        if self.check_availability(row, seat):
            self.seating_plan[row - 1][seat - 1] = SeatStatus.BOOKED
            print(f"Ticket booked successfully for Row {row}, Seat {seat}")
            self.notify_observers()
        else:
            print(f"Seat {row}-{seat} is already booked")

    def cancel_booking(self, row, seat):
        # Скасування бронювання та сповіщення спостерігачів
        if 1 <= row <= self.rows and 1 <= seat <= self.seats_per_row:
            if self.seating_plan[row - 1][seat - 1] == SeatStatus.BOOKED:
                self.seating_plan[row - 1][seat - 1] = SeatStatus.AVAILABLE
                print(f"Booking canceled for Row {row}, Seat {seat}")
                self.notify_observers()
            else:
                print(f"No booking found for Seat {row}-{seat}")
        else:
            raise ValueError("Invalid row or seat number")

# Клас спостерігача для виводу в консоль
class ConsoleObserver(Observer):
    def init(self, subject):
        # Приєднання спостерігача до системи бронювання
        self.subject = subject
        self.subject.attach(self)

    def update(self):
        # Оновлення та виведення плану місць у консоль
        print("Seating plan has been updated:")
        self.subject.display_seating_plan()

# Приклад використання
ticket_system = TicketBookingSystem(rows=5, seats_per_row=10)
console_observer = ConsoleObserver(ticket_system)

try:
    ticket_system.book_ticket(2, 5)
    ticket_system.book_ticket(3, 7)
    ticket_system.cancel_booking(2, 5)

except ValueError as e:
    print(f"Error: {e}")